In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import os 
import numpy as np
import pandas as pd 
import seaborn as sns 
from pandas.api.types import is_numeric_dtype

In [8]:
path= '../../datasets/main_data/bank-additional-full.csv'
full_bank = pd.read_csv(path, sep=';')

#### Module Data.py codes

In [9]:
#%%writefile data.py
#%%writefile ../scripts/project_package/data_package/data.py

import seaborn as sns
import matplotlib.pyplot as plt 
import os
import numpy as  np
import pandas as pd
import pandas
from pandas.api.types import is_numeric_dtype
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.base import *


class WrangleData():
    
    def __repr__(self):
        
        return "Used to prepare data for wrangling"
    
    def  __init__(self):
        
        pass
        
    
    
    def load_data(self, path="", sep=",", cols_to_drop=[]):
                
        self.path = path
        self.cols_to_drop = cols_to_drop 
        self.sep = sep  
            
        try :
            self.data = pd.read_csv(path, sep)
            
            if len(self.cols_to_drop) > 0:
                for col in self.cols_to_drop:
                    self.data.drop(col, axis=1, inplace=True)
                        
            self._fit() 
            
            return self.data 
        
        except:
            
            "No data path was passed upon call of method load_data"
    
    def _fit(self):
        
        try:
            assert(type(self.data) is pandas.core.frame.DataFrame), "data must be of type pandas.DataFrame"
            
            print("You are now fit to use this object for wrangling")
                    
        except AttributeError:
            
            print("Hey Buddy you need to load a data first !!! ")
        

    def get_data(self):
        
        return self.data

    def check_outliers(self, show_plot=False, save_img=os.getcwd()+'/outliers.png'):
 
        """
        This functions checks for columns with outlers using the IQR method

        It accespts as argmuent a dataset. 
        show_plot can be set to True to output pairplots of outlier columns    
        """

        self.outliers = [] 
        Q1 = self.data.quantile(0.25)  
        Q3 = self.data.quantile(0.75)
        IQR = Q3 - Q1
        num_data = self.data.select_dtypes(include='number')
        result = dict ((((num_data < (Q1 - 1.5 * IQR)) | (num_data > (Q3 + 1.5 * IQR)))==True).any())
        #data[(data[col] >= high)|(data[col] <= low)].index
        index = self.data[(num_data < Q1 - 1.5 * IQR) | (num_data > Q3 + 1.5 * IQR)].index
        for k,v in result.items():
            if v == True:  
                self.outliers.append(k)
        if show_plot:
            self.outlier_pair_plot = sns.pairplot(self.data[self.outliers]);
            print(f'{result},\n\n Visualization of outlier columns')
            plt.savefig(fname=save_img, format='png')
            return  self.outlier_pair_plot
        else:
            return self.data.loc[index, self.outliers] 
        
                
                
        
        
    def treat_outliers(self, type_='median_replace'):
            
        """
        This treat outliers using any ofthses 3 methods as specified by user

            1. median_replace -  median replacement

            2. quant_floor - quantile flooring

            3. trim - trimming 

            4. log_transform - log transformations
            
            5. isf    -       IsolationForest (also like trimming)

        The methods are some of the commont statistical methods in treating outler
        columns

        By default treatment type is set to median replacement

        """

        if type_ == "median_replace":

            for col in self.data.columns.tolist():
                if is_numeric_dtype(self.data[col]):
                    median = (self.data[col].quantile(0.50))
                    q1 = self.data[col].quantile(0.25)
                    q3 = self.data[col].quantile(0.75)
                    iqr = q3 - q1
                    high = int(q3 + 1.5 * iqr) 
                    low = int(q1 - 1.5 * iqr)
                    self.data[col] = np.where(self.data[col] > high, median, self.data[col])
                    self.data[col] = np.where(self.data[col] > high, median, self.data[col])        

        if type_ == "quant_floor":

            for col in self.data.columns.tolist():
                if is_numeric_dtype(data[col]):
                    q_10 = self.data[col].quantile(0.5)
                    q_90 = self.data[col].quantile(0.95)
                    self.data[col] =  self.data[col] = np.where(self.data[col] < q_10, q_10 , self.data[col])
                    self.data[col] =  self.data[col] = np.where(self.data[col] > q_90, q_90 , self.data[col])

        if type_ == "trim": 

            for col in self.data.columns.tolist():
                low = .05
                high = .95
                quant_df = self.data.quantile([low, high])
                for name in list(self.data.columns):
                    if is_numeric_dtype(self.data[name]):
                        self.data = self.data[(self.data[name] >= quant_df.loc[low, name]) 
                            & (self.data[name] <= quant_df.loc[high, name])]

        if type_ == "log_transform":  
            for col in self.data.columns.tolist():
                if is_numeric_dtype(self.data[col]):
                    self.data[col] = self.data[col].map(lambda i: np.log(i) if i > 0 else 0)

        if type_ == "isf":
            iso = IsolationForest(contamination=0.1)
            yhat = iso.fit_predict(self.data.select_dtypes(exclude='object'))
            #select all rows that are not outliers
            mask = yhat != -1 
            self.data = self.data[mask]


        return self.data  
    
    
    def map_col_values(self, col_name="", values_dict={}):
        
        """
        replace values in a series (values_dict.keys) with specified values from (values_dict.values)
        """

        self.data[col_name] = self.data[col_name].map(values_dict)

        return self.data
    
    
    def split_data_single(self, target_cols=[]):
            
        self.split1 = self.data.drop(columns=target_cols, axis=1) 

        self.split2   = pd.DataFrame(self.data[target_cols])

        return self.split1, self.split2
    
    
    def encode (self, use_split1=False, use_split2 = False, use_data=False): 
        
        if use_data:
      
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
            to_encode = self.data.select_dtypes(exclude='number')
            if self.data.shape[1] > 1:
                #ohe = MultiLabelBinarizer()
                self.data.drop(to_encode.columns.tolist(), axis=1, inplace = True)
                features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
                self.data = self.data.merge(features_cat_encode, left_index=True, right_index=True)
               # print(ohe.classes_) 
            else: 
                self.data = pd.DataFrame(ohe.fit_transform(to_encode))
                print(ohe.categories_) 

            return self.data
        
        
        if use_split1:
      
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
            to_encode = self.split1.select_dtypes(exclude='number')
            if self.split1.shape[1] > 1:
                #ohe = MultiLabelBinarizer()
                self.split1.drop(to_encode.columns.tolist(), axis=1, inplace = True)
                features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
                self.split1 = self.split1.merge(features_cat_encode, left_index=True, right_index=True)
               # print(ohe.classes_) 
            else: 
                self.split1 = pd.DataFrame(ohe.fit_transform(to_encode))
                print(ohe.categories_) 

            return self.split1
        
        
        if use_split2:
      
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
            to_encode = self.split2.select_dtypes(exclude='number')
            if self.split2.shape[1] > 1:
                #ohe = MultiLabelBinarizer()
                self.split2.drop(to_encode.columns.tolist(), axis=1, inplace = True)
                features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
                self.split2 = self.split2.merge(features_cat_encode, left_index=True, right_index=True)
               # print(ohe.classes_) 
            else: 
                self.split2 = pd.DataFrame(ohe.fit_transform(to_encode))
                print(ohe.categories_) 

            return self.split2
            

    def scale_data(self, scaler=RobustScaler(),
                  use_data=False, use_split1= False, use_split2 = False):
        
        """
            Specify scaler type, scaler type must have fit_transform as a method
        """
        
        if use_data:
            self.data = scaler.fit_transform(self.data)
            return self.data 
        
        if use_split1:
            self.split1 = scaler.fit_transform(self.split1)
            return self.split1
        
        if use_split2:
            self.split2 = scaler.fit_transform(self.split2)
            return self.split2
            

### Test data.py module codes

In [10]:
wrangle = WrangleData()

In [11]:
wrangle.load_data(path, sep=';')

You are now fit to use this object for wrangling


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [12]:
wrangle.data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [13]:
wrangle.check_outliers()

,age,duration,campaign,pdays,previous,cons.conf.idx
0,56,261,1,999,0,-36.4
1,57,149,1,999,0,-36.4
2,37,226,1,999,0,-36.4
3,40,151,1,999,0,-36.4
4,56,307,1,999,0,-36.4
...,...,...,...,...,...,...
41183,73,334,1,999,0,-50.8
41184,46,383,1,999,0,-50.8
41185,56,189,2,999,0,-50.8
41186,44,442,1,999,0,-50.8


In [14]:
wrangle.treat_outliers(type_="isf")

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40544,33,blue-collar,married,professional.course,no,yes,no,cellular,aug,tue,...,4,999,0,nonexistent,-1.7,94.027,-38.3,0.886,4991.6,yes
40545,31,admin.,single,high.school,no,no,no,cellular,aug,tue,...,1,999,1,failure,-1.7,94.027,-38.3,0.886,4991.6,no
40549,58,retired,divorced,basic.4y,no,no,no,telephone,aug,tue,...,1,999,0,nonexistent,-1.7,94.027,-38.3,0.886,4991.6,no
40550,34,blue-collar,married,basic.9y,no,yes,no,cellular,aug,tue,...,2,999,0,nonexistent,-1.7,94.027,-38.3,0.886,4991.6,no


In [15]:
wrangle.split_data_single(target_cols=['duration']);

In [16]:
wrangle.split2.head(2)

,duration
0,261
1,149


In [17]:
wrangle.split1.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [18]:
wrangle.encode(use_split1 = True)

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,0,...,44,45,46,47,48,49,50,51,52,53
0,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,57,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,37,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,40,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37063,42,1,999,0,-2.9,92.469,-33.6,1.029,5076.2,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
37064,42,1,999,0,-2.9,92.469,-33.6,1.029,5076.2,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
37065,32,1,999,0,-2.9,92.469,-33.6,1.029,5076.2,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
37067,42,2,999,0,-2.9,92.469,-33.6,1.029,5076.2,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [19]:
wrangle.split1

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,0,...,44,45,46,47,48,49,50,51,52,53
0,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,57,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,37,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,40,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37063,42,1,999,0,-2.9,92.469,-33.6,1.029,5076.2,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
37064,42,1,999,0,-2.9,92.469,-33.6,1.029,5076.2,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
37065,32,1,999,0,-2.9,92.469,-33.6,1.029,5076.2,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
37067,42,2,999,0,-2.9,92.469,-33.6,1.029,5076.2,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [20]:
wrangle.scale_data(use_split2=True)

array([[ 0.42512077],
       [-0.11594203],
       [ 0.25603865],
       ...,
       [-0.67149758],
       [ 0.60386473],
       [-0.70531401]])

In [21]:
wrangle.split2

array([[ 0.42512077],
       [-0.11594203],
       [ 0.25603865],
       ...,
       [-0.67149758],
       [ 0.60386473],
       [-0.70531401]])

In [22]:
"Works Fine"

'Works Fine'

#### Module univariate_plot.py codes

In [23]:
#%%writefile uni_plot.py
#%%writefile ../scripts/project_package/plot_package/uni_plot.py

import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import *
import os
import numpy as np 
import pandas as pd
from pandas.api.types import is_numeric_dtype

def plot_univariate (data, x=None, y=None, color='r',save=False,
                title='New Chart', chart_type='hist', xlabel='', ylabel='',
                    save_to=os.getcwd(), log_normalise=False):
    
    
    """
    Make a univariate plot of any of these selcted types:
    
    1. bar - barchart
    
    2. hist - Histogram
    
    3. pie - Piechart
    
    4. count - Countplot
    
    
    """
    
    plt.subplots(figsize=(10,7))
    plt.title(title, fontsize=18)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel(ylabel, fontsize=15)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    
    if chart_type == 'hist':
        if log_normalise:
            data = np.log(data)
        plot = sns.distplot(a=data, color=color)
        if save:
            plt.savefig(fname=save_to+f'/{title}.png', format='png')
        
    return plot

#### Testing univariate_plot.py module codes

#### Model bivariate_plot.py codes

In [24]:
#%%writefile bi_plot.py
#%%writefile ../scripts/project_package/plot_package/bi_plot.py

import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import *
import os
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype


def plot_bivariate(data, x=None, y=None, hue=None, 
                  color='r',save=False,
                title='New Chart', chart_type='hist',
                   xlabel='', ylabel='',
                    save_to=os.getcwd(), img_name = " ", 
                   palette={'use':False, "size":1}, log_normalise=False,
                  kind_joint_plot = 'scatter', kind_pair_plot="scatter", figsize=(10,7)):
    
    """
    Make a bivariate plot of any of the selcted types:
    
    1. bar - barchart
    
    2. scatter  - scatter plot
    
    3. cat  - catplot
    
    4. count - countplot
    
    5 joint - jointplot 
    
    6  pair - pairplot
    
    7  corr - corr_plot
    
    When calling joint_plot:
        
        kind_joint_plot is default to `scatter`
        other types include "reg", "reside", "kde", "hex"
        
    When calling pair_plot:
        
        kind_pair_plot is default to `scatter`
        other types include 'reg'
    """
    def plt_tweaks():
        plt.subplots(figsize= figsize)
        plt.title(title, fontsize=18)
        plt.xlabel(xlabel, fontsize=15)
        plt.ylabel(ylabel, fontsize=15)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
    
    
    # define helper functions
    
    def use_palette():
        palettes = []
#        palette_to_use=[]
        if palette['use'] == True:
            palette_to_use = [palettes[i] for i in range(palette['size'])]
            
            return palette_to_use

    def log_norm():
        if log_normalise and y != None:
            y = np.log(y)
        elif log_normalise and y == None:
            data = np.log(data)
            
    def save_image():
        if save:
            if img_name != " ":
                plt.savefig(fname=save_to+"/"+img_name+'.png', format='png')
            else:
                plt.savefig(fname=save_to+f'/{title}.png', format='png')
                
        
    # make plots
    
    if chart_type == "joint":
        log_norm()
        plot = sns.jointplot(x=x, y=y, data=data,
                            height=6, ratio=5, space=0.2, kind=kind_joint_plot)
        
        save_image()
        
    if chart_type == "pair":
       # try:
        log_norm()
        if palette['use'] == True:
            palette_to_use = use_palette()
            plot = sns.pairplot(data, palette=palette_to_use, 
                            kind= kind_pair_plot,height=3, aspect=1, hue=hue)
        else:
             plot = sns.pairplot(data, 
                            kind= kind_pair_plot,height=2.5, aspect=1, hue=hue, )
        save_image()
        
    if chart_type  == "corr":
        plt_tweaks()
        corr_data = data.corr()
        corr_plot = sns.heatmap(corr_data,annot=True, fmt='.2g', center=0) 
        
    return plot

#### Testing bivariate_plot.py module codes

#### Module model.py codes

In [25]:
#%%writefile model.py
#%%writefile ../scripts/project_package/model_package/model.py


import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from sklearn.linear_model import *
from imblearn.over_sampling import *
from imblearn.pipeline import *

from imblearn.metrics import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.base import *
from sklearn.model_selection import * 

def plot_pca_components(data):
    pca = PCA().fit(data)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('number of components')
    plt.ylabel('cumulative explained variance');
    
def check_imbalance(data,label='', x=0.7, y=30000):
    plt.subplots(figsize=(10,8)) 
    data[label].value_counts().plot(kind='bar')
    text = f'Class Imbalance Count:\n\n{data[label].value_counts().to_dict()}'
    plt.text(x=x, y=y, s = text ,  fontsize=15)
    
def encode (data):
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
    to_encode = data.select_dtypes(exclude='number')
    if data.shape[1] > 1:
        #ohe = MultiLabelBinarizer()
        data.drop(to_encode.columns.tolist(), axis=1, inplace = True)
        features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
        data = data.merge(features_cat_encode, left_index=True, right_index=True)
        #print(ohe.classes_) 
    else:
        data = pd.DataFrame(ohe.fit_transform(to_encode))
        print(ohe.categories_) 
    return data 


def x_y_split(data, x=None, y=None, type_="single", test_size=.10):
    
    """
    Single type divides into just x and y
    Double type divides into train and test for each of x and y
    """
    
    X, y = data.drop(columns=y, axis=1), data[y]
    
    if type_ == "single":
        
        return X, y
    
    if type == "double":
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                               test_size=test_size, random_state=123)
        
        return X_train, X_test, y_train, y_test
    
    
def gridSearch(model,hyper_params={},cv=StratifiedKFold(), x_train=None, y_train=None):
    
    """
    Performs GridSeach of the best hyperparmaters for the passed model
    """
    
    search = GridSearchCV(model=model, param_grid = hyper_params, n_jobs=-1, cv=cv)
    search.fit(X=x_train, y=y_train)
    print("Best parameter (CV score=%0.3f):\n" % search.best_score_)
    print(search.best_params_)
    print(search.score) 
    return search


def plot_grid_search(search_obj, pca_obj, X_train):
    
    """
    Prints the best (optimised) hyperparmatersfor the grid search object
    and plots the optimised pca components
    """
    
    print("Best parameter (CV score=%0.3f):\n" % search.best_score_)
    print("Best Params:",search.best_params_)
    pca.fit(X_train_scaled)

    fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(8, 8))
    ax0.plot(np.arange(1, pca.n_components_ + 1),
             pca.explained_variance_ratio_, '+', linewidth=2)
    ax0.set_ylabel('PCA explained variance ratio')

    ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
                linestyle=':', label='n_components chosen')
    ax0.legend(prop=dict(size=12))

    # For each number of components, find the best classifier results
    results = pd.DataFrame(search.cv_results_)
    components_col = 'param_pca__n_components'
    best_clfs = results.groupby(components_col).apply(
        lambda g: g.nlargest(1, 'mean_test_score'))

    best_clfs.plot(x=components_col, y='mean_test_score', yerr='std_test_score',
                   legend=False, ax=ax1)
    ax1.set_ylabel('Classification accuracy (val)')
    ax1.set_xlabel('n_components')

    plt.xlim(-1, 70)

    plt.tight_layout()
    plt.show() 
    
    
class Preprocessor(BaseEstimator, TransformerMixin):
    
    def __repr__(self):
        
        return "Used to prepare data for modelling"
    
    def  __init__(self):
        
        pass
        
    
    def fit(self, data, y=None):
        
        assert(type(data) is pandas.core.frame.DataFrame), "data must be of type pandas.DataFrame"
        
        self.data = data 
        
        print("Fitted")
        
        return self 
        


    def check_outliers(self, show_plot=False, save_img=os.getcwd()+'/outliers.png'):
            
        """
        This functions checks for columns with outlers using the IQR method

        It accespts as argmuent a dataset. 
        show_plot can be set to True to output pairplots of outlier columns    
        """

        self.outliers = [] 
        Q1 = self.data.quantile(0.25)  
        Q3 = self.data.quantile(0.75)
        IQR = Q3 - Q1
        num_data = self.data.select_dtypes(include='number')
        result = dict ((((num_data < (Q1 - 1.5 * IQR)) | (num_data > (Q3 + 1.5 * IQR)))==True).any())
        #data[(data[col] >= high)|(data[col] <= low)].index
        index = self.data[(num_data < Q1 - 1.5 * IQR) | (num_data > Q3 + 1.5 * IQR)].index
        for k,v in result.items():
            if v == True:  
                self.outliers.append(k)
        if show_plot:
            self.outlier_pair_plot = sns.pairplot(self.data[self.outliers]);
            print(f'{result},\n\n Visualization of outlier columns')
            plt.savefig(fname=save_img, format='png')
            return  self.outlier_pair_plot
        else:
            return self.data.loc[index, self.outliers] 
        
        
    def treat_outliers(self, type_='median_replace'):
            
        """
        This treat outliers using any ofthses 3 methods as specified by user

            1. median_replace -  median replacement

            2. quant_floor - quantile flooring

            3. trim - trimming 

            4. log_transform - log transformations

        The methods are some of the commont statistical methods in treating outler
        columns

        By default treatment type is set to median replacement

        """

        if type_ == "median_replace":

            for col in self.data.columns.tolist():
                if is_numeric_dtype(self.data[col]):
                    median = (self.data[col].quantile(0.50))
                    q1 = self.data[col].quantile(0.25)
                    q3 = self.data[col].quantile(0.75)
                    iqr = q3 - q1
                    high = int(q3 + 1.5 * iqr) 
                    low = int(q1 - 1.5 * iqr)
                    self.data[col] = np.where(self.data[col] > high, median, self.data[col])
                    self.data[col] = np.where(self.data[col] > high, median, self.data[col])        

        if type_ == "quant_floor":

            for col in self.data.columns.tolist():
                if is_numeric_dtype(data[col]):
                    q_10 = self.data[col].quantile(0.5)
                    q_90 = self.data[col].quantile(0.95)
                    self.data[col] =  self.data[col] = np.where(self.data[col] < q_10, q_10 , self.data[col])
                    self.data[col] =  self.data[col] = np.where(self.data[col] > q_90, q_90 , self.data[col])

        if type_ == "trim": 

            for col in self.data.columns.tolist():
                low = .05
                high = .95
                quant_df = self.data.quantile([low, high])
                for name in list(self.data.columns):
                    if is_numeric_dtype(self.data[name]):
                        self.data = self.data[(self.data[name] >= quant_df.loc[low, name]) 
                            & (self.data[name] <= quant_df.loc[high, name])]

        if type_ == "log_transform":  
            for col in self.data.columns.tolist():
                if is_numeric_dtype(self.data[col]):
                    self.data[col] = self.data[col].map(lambda i: np.log(i) if i > 0 else 0)

        if type_ == "isf":
            iso = IsolationForest(contamination=0.1)
            yhat = iso.fit_predict(self.data.select_dtypes(exclude='object'))
            #select all rows that are not outliers
            mask = yhat != -1 
            self.data = self.data[mask]


        return self.data 
    
    
    def map_col_values(self, col_name="", values_dict={}):

        self.data[col_name] = self.data[col_name].map(values_dict)

        return self.data
    
    
    def split_data_single(self, target_cols=[]):
            
        self.features = self.data.drop(columns=target_cols, axis=1) 

        self.target   = pd.DataFrame(self.data[target_cols])

        return self.features, self.target
    
    
    def encode (self, data_obj=None, use_features=True, use_target=False): 
        
        if data_obj is None and use_features == False and use_target == False:
        
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
            to_encode = self.data.select_dtypes(exclude='number')
            if self.data.shape[1] > 1:
                #ohe = MultiLabelBinarizer()
                self.data.drop(to_encode.columns.tolist(), axis=1, inplace = True)
                features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
                self.data = self.data.merge(features_cat_encode, left_index=True, right_index=True)
               # print(ohe.classes_) 
            else: 
                self.data = pd.DataFrame(ohe.fit_transform(to_encode))
                print(ohe.categories_) 
            return self.data
        
        if data_obj is not None:
        
            self.data_obj = data_obj
            print("Not None")
            
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
            to_encode = self.data_obj.select_dtypes(exclude='number')
            if self.data_obj.shape[1] > 1:
                #ohe = MultiLabelBinarizer()
                self.data_obj.drop(to_encode.columns.tolist(), axis=1, inplace = True)
                features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
                self.data_obj = self.data_obj.merge(features_cat_encode, left_index=True, right_index=True)
               # print(ohe.classes_) 
            else:
                self.data_obj = pd.DataFrame(ohe.fit_transform(to_encode))
                print(ohe.categories_) 
            return self.data_obj
        
        if use_features:
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
            to_encode = self.features.select_dtypes(exclude='number')
            if self.features.shape[1] > 1:
                #ohe = MultiLabelBinarizer()
                self.features.drop(to_encode.columns.tolist(), axis=1, inplace = True)
                features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
                self.features = self.features.merge(features_cat_encode, left_index=True, right_index=True)
               # print(ohe.classes_) 
            else: 
                self.features = pd.DataFrame(ohe.fit_transform(to_encode))
                print(ohe.categories_) 
            return self.features
        
        if use_target:
            
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', )
            to_encode = self.target.select_dtypes(exclude='number')
            if self.target.shape[1] > 1:
                #ohe = MultiLabelBinarizer()
                self.target.drop(to_encode.columns.tolist(), axis=1, inplace = True)
                features_cat_encode = pd.DataFrame(ohe.fit_transform(to_encode))
                self.target = self.target.merge(features_cat_encode, left_index=True, right_index=True)
               # print(ohe.classes_) 
            else: 
                self.target = pd.DataFrame(ohe.fit_transform(to_encode))
                print(ohe.categories_) 
            return self.target
            
    
    def split_data_double(self, features_=pd.DataFrame([[]]), target_=pd.DataFrame([[]]), 
                          test_size=.10, use_native=True):
        
        if use_native == False:
        
            if features.shape[0] != target.shape[0]:
                
                raise Exception("Wrong, you are trying to pass unequal shapes\n\
                Shapes of dataframes must be equal\n\
                Try target = target.iloc[0:features.shape[0]]")

            self.features_ = features_
            self.target_ = target_

            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.features_, 
                                                                                    self.target_,
                                           test_size= test_size, random_state=24)

            return self.X_train, self.X_test, self.y_train, self.y_test
        
        if use_native:
            
            self.target = self.target.iloc[0:self.features.shape[0]]
            
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.features, self.target,
                                           test_size= test_size, random_state=24)

            return self.X_train, self.X_test, self.y_train, self.y_test
        
    
    
    


    def scale_data(self, scale_data=pd.DataFrame([[]]),
                   scaler=RobustScaler(), use_features=True,
                  use_target=False, use_data=False):
        
        """
            Specify scaler type, scaler type must have fit_transform as a method
        """
        
        if use_features:
    
            self.features = scaler.fit_transform(self.features)

            return self.features
        
        if use_target:
            
            self.target = scaler.fit_transform(self.target)

            return self.target
        
        if use_data:
            
            self.data = scaler.fit_transform(self.data)

            return self.data
        
        if use_data == False and use_features == False and use_target == False:
            
            self.scale_data = scale_data
            
            self.scale_data = scaler.fit_transform(self.scale_data)

            return self.scale_data
            
    def transform(self, X):
        
        """
        Ideally, a preapred trainX data ought to be passed to in case of passing into a pipeline
        """
        
        self.data = X
                
        self.data = self.treat_outliers(type_="isf") 
        
        #self.data = self.map_col_values(col_name="y", values_dict={"no":0, "yes":1})
        
       # self.features, self.target = self.split_data_single(target_cols=["y"])
        #print(self.features)
                
        self.features = self.encode(self.features)
      #  self.target = self.target.iloc[0:self.features.shape[0], 0:]
        #print(self.target)
       # self.X_train, self.X_test, self.y_train, self.y_test = self.split_data_double(
        #    self.features, self.target, test_size=.10)
        
        scaler=RobustScaler() 
            
        X = scaler.fit_transform(self.X_train)
        
        return X
    
    
    def fit_transform(self, X, y=None):
        
        self.X = X
        
        return self.transform(self.X) 

### Testing model.py codes

In [26]:
p = Preprocessor() 

In [29]:
data = wrangle.data 

In [30]:
data.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [31]:
p.fit(data)

Fitted


Used to prepare data for modelling

In [32]:
data.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [33]:
p.check_outliers()

,age,duration,campaign,pdays,previous
0,56,261,1,999,0
1,57,149,1,999,0
2,37,226,1,999,0
3,40,151,1,999,0
4,56,307,1,999,0
...,...,...,...,...,...
40544,33,253,4,999,0
40545,31,241,1,999,1
40549,58,34,1,999,0
40550,34,298,2,999,0


In [34]:
treated_data = p.treat_outliers(type_='isf')

In [35]:
 treated_data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36809,36,self-employed,single,university.degree,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,yes
36814,34,housemaid,single,basic.4y,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,no
36820,30,services,single,high.school,no,yes,no,cellular,jun,mon,...,2,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,yes
36847,37,technician,married,professional.course,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,yes


In [36]:
p.map_col_values(col_name='y', values_dict={'yes':1, 'no':0})

/home/patrick/.local/lib/python3.6/site-packages/ipykernel_launcher.py:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36809,36,self-employed,single,university.degree,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1
36814,34,housemaid,single,basic.4y,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,0
36820,30,services,single,high.school,no,yes,no,cellular,jun,mon,...,2,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1
36847,37,technician,married,professional.course,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1


In [37]:
p.data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36809,36,self-employed,single,university.degree,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1
36814,34,housemaid,single,basic.4y,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,0
36820,30,services,single,high.school,no,yes,no,cellular,jun,mon,...,2,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1
36847,37,technician,married,professional.course,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1


In [38]:
p.data.rename(columns={'y':'purchases'}, inplace=True)

/home/patrick/.local/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
p.data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,purchases
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36809,36,self-employed,single,university.degree,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1
36814,34,housemaid,single,basic.4y,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,0
36820,30,services,single,high.school,no,yes,no,cellular,jun,mon,...,2,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1
36847,37,technician,married,professional.course,no,no,no,cellular,jun,mon,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2,1


In [40]:
p.split_data_single(target_cols=['purchases'])

(       age            job  marital            education  default housing loan  \
 0       56      housemaid  married             basic.4y       no      no   no   
 1       57       services  married          high.school  unknown      no   no   
 2       37       services  married          high.school       no     yes   no   
 3       40         admin.  married             basic.6y       no      no   no   
 4       56       services  married          high.school       no      no  yes   
 ...    ...            ...      ...                  ...      ...     ...  ...   
 36809   36  self-employed   single    university.degree       no      no   no   
 36814   34      housemaid   single             basic.4y       no      no   no   
 36820   30       services   single          high.school       no     yes   no   
 36847   37     technician  married  professional.course       no      no   no   
 36849   35     management  married    university.degree       no      no   no   
 
          cont

In [41]:
p.features

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36809,36,self-employed,single,university.degree,no,no,no,cellular,jun,mon,177,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2
36814,34,housemaid,single,basic.4y,no,no,no,cellular,jun,mon,157,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2
36820,30,services,single,high.school,no,yes,no,cellular,jun,mon,181,2,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2
36847,37,technician,married,professional.course,no,no,no,cellular,jun,mon,129,1,999,0,nonexistent,-2.9,92.963,-40.8,1.260,5076.2


In [42]:
p.target

,purchases
0,0
1,0
2,0
3,0
4,0
...,...
36809,1
36814,0
36820,1
36847,1


In [43]:
p.encode()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,42,43,44,45,46,47,48,49,50,51
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33357,33,277,1,999,0,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
33358,35,200,3,999,0,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
33359,48,725,1,999,0,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
33360,33,204,1,999,1,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [44]:
p.features

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,42,43,44,45,46,47,48,49,50,51
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33357,33,277,1,999,0,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
33358,35,200,3,999,0,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
33359,48,725,1,999,0,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
33360,33,204,1,999,1,-1.8,92.893,-46.2,1.291,5099.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
p.split_data_double() 

(       age  duration  campaign  pdays  previous  emp.var.rate  cons.price.idx  \
 14563   59        85         5    999         0           1.4          93.918   
 17144   35       102         2    999         0           1.4          93.918   
 8675    32       135         3    999         0           1.4          94.465   
 19499   35       111         1    999         0           1.4          93.444   
 13767   23       119         2    999         0           1.4          93.918   
 ...    ...       ...       ...    ...       ...           ...             ...   
 20568   32       222         2    999         0           1.4          93.444   
 26290   56       110         1    999         0          -0.1          93.200   
 15048   31        84         1    999         0           1.4          93.918   
 905     32       245         1    999         0           1.1          93.994   
 13206   28        85         1    999         0           1.4          93.918   
 
        cons.c

In [46]:
p.X_train

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,42,43,44,45,46,47,48,49,50,51
14563,59,85,5,999,0,1.4,93.918,-42.7,4.961,5228.1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
17144,35,102,2,999,0,1.4,93.918,-42.7,4.962,5228.1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8675,32,135,3,999,0,1.4,94.465,-41.8,4.864,5228.1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
19499,35,111,1,999,0,1.4,93.444,-36.1,4.968,5228.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13767,23,119,2,999,0,1.4,93.918,-42.7,4.963,5228.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20568,32,222,2,999,0,1.4,93.444,-36.1,4.966,5228.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
26290,56,110,1,999,0,-0.1,93.200,-42.0,4.076,5195.8,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
15048,31,84,1,999,0,1.4,93.918,-42.7,4.958,5228.1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
905,32,245,1,999,0,1.1,93.994,-36.4,4.856,5191.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


#### Train Models

#### Models to be used

- LogisticRegression (CV)

- RandomForest Classifier

- SVM

- XGboost

- MLP (Multi-Layer Perceptron Network)

In [47]:
from sklearn.pipeline import FeatureUnion, Pipeline  
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline, make_pipeline
from sklearn.decomposition import PCA
from imblearn.metrics import make_index_balanced_accuracy
from sklearn.metrics import balanced_accuracy_score

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.tree import ExtraTreeClassifier
import xgboost
from sklearn.neural_network import MLPClassifier

##### model1 - Logistic Regression CV

In [115]:
model1 = make_pipeline(SMOTE(sampling_strategy=.60), 
                       PCA(n_components=45), LogisticRegressionCV(cv=5,
                                                                  random_state=123, max_iter=1000));

In [116]:
model1.fit(p.X_train,p.y_train.purchases.ravel());

##### Test model1

In [117]:
model1_pred = model1.predict(p.X_test)

In [118]:
model1_pred

array([0, 0, 0, ..., 0, 0, 0])

In [119]:
balanced_accuracy_score(p.y_test, model1_pred)

0.858125880277779

##### Train model2 - Logistic Regression

In [124]:
model2 = make_pipeline(SMOTE(sampling_strategy=.60), PCA(n_components=45), LogisticRegression(random_state=124
                                                                                             ,max_iter=1000))

In [125]:
model2.fit(p.X_train,p.y_train.purchases.ravel());

/home/patrick/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


##### Test Model2

In [126]:
model2_pred = model2.predict(p.X_test)
model2_pred

array([0, 0, 0, ..., 0, 0, 0])

In [127]:
balanced_accuracy_score(p.y_test, model2_pred) 

0.8622279191899445

##### Train Model3 - RandomForest Classifier (SkLearn Implementation)

In [128]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
model3 = make_pipeline(SMOTE(sampling_strategy=.60), PCA(n_components=45), RandomForestClassifier())
model3.fit(p.X_train,p.y_train.purchases.ravel()); 

##### Test Model3

In [130]:
model3_pred = model3.predict(p.X_test)
balanced_accuracy_score(p.y_test, model3_pred) 

0.7535722061038517

##### Train Model4 - RandomForest Classifier (Imblearn Implementation)

In [131]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [132]:
model4 = make_pipeline(SMOTE(sampling_strategy=.60), PCA(n_components=45), BalancedRandomForestClassifier())
model4.fit(p.X_train,p.y_train.purchases.ravel());

##### Test Model3

In [133]:
model4_pred = model4.predict(p.X_test)
model4_pred_proba = model4.predict_proba(p.X_test)

> Probability of each input belonging to a particular class (purchase and no-purchase)

In [134]:
model4_pred_proba

array([[0.85, 0.15],
       [0.7 , 0.3 ],
       [1.  , 0.  ],
       ...,
       [1.  , 0.  ],
       [0.96, 0.04],
       [1.  , 0.  ]])

> model score

In [135]:
balanced_accuracy_score(p.y_test, model4_pred)  

0.8289217624660663

> The BalancedRandomForestClassifier Implementation performs better than  RandomForestClassifier

##### Train Model5 - SVM Classification

In [136]:
from sklearn.svm import SVC

In [137]:
model5 = make_pipeline(SMOTE(sampling_strategy=.60), PCA(n_components=45), SVC())
model5.fit(p.X_train,p.y_train.purchases.ravel());

Test Model5

In [138]:
model5_pred = model5.predict(p.X_test)

In [139]:
balanced_accuracy_score(p.y_test, model5_pred)  

0.829394946483554

#### Module model_metrics.py codes

In [215]:
# %%writefile model_metrics.py
#%%writefile ../scripts/project_package/model_package/model_metrics.py

from imblearn.metrics import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.base import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

import pandas as pd

class Metrics ():
    
    def __init__(self, X_train, y_train, X_test, y_test, y_hat=np.array([])):
        
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.y_hat =  y_hat
        
    
    def class_report(self):
        
        full_report = classification_report_imbalanced(self.y_test, self.y_hat)
        
        print(full_report)
        
    def conf_matrix(self):
        
        conf_matrix = confusion_matrix(self.y_test, self.y_hat)
        
        conf_matrix_df = pd.DataFrame(conf_matrix, columns=['Actual_+ve', 'Actual_-ve'],
                               index=['predicted_+ve', 'predicted_-ve'])
        
        return conf_matrix_df
    
    def accuracy_score(self):
        return  accuracy_score(self.y_test, self.y_hat)
    
    def classification_error(self):
        
        return 1 - accuracy_score() 
        
    def specif_sensitiv(self):
        
        """
        Sensitivity: When the actual value is positive, how often is the prediction correct?
        
        Specificity: When the actual value is negative, how often is the prediction correct?
        """
        
        conf_matrix = confusion_matrix(self.y_test, self.y_hat)
        
        TP = conf_matrix[1, 1]
        TN = conf_matrix[0, 0]
        FP = conf_matrix[0, 1]
        FN = conf_matrix[1, 0]
        
        sensitivity = TP / float(FN + TP)
        specificity = TN / (TN + FP)
        
        sensitiv_specific_table = pd.DataFrame([[sensitivity, specificity]],
                                               columns=['sensitivity', 'specificity'])
        
        return sensitiv_specific_table
    
    
    def evaluate_classifier(self, clf, models_eval_scores, clf_name=None): # X_train, y_train, X_test, y_test, ):
    
        self.clf = clf
        self.models_eval_scores = models_eval_scores
        self.clf_name = clf_name
        
        """
        Returns a dataframe of unbalanced and balanced acuuracy score of estimators used
        Run for the first time, pass an empty dataframe of df_scorees 
        and when running on more estimators, pass the previous df_scores dataframe for a 
        single table of evaluation scores

        Example given below:

                        LogisticRegressionCV
                Accuracy 	        0.908
                Balanced accuracy 	0.842

        """
        from imblearn.pipeline import Pipeline as ImbPipe
        from sklearn.pipeline import Pipeline as Pipe
        if self.clf_name is None:
            if isinstance(self.clf, ImbPipe) or isinstance(self.clf, Pipe):
                self.clf_name = self.clf[-1].__class__.__name__
                print(self.clf_name)
            else:
                self.clf_name = self.clf.__class__.__name__
                print(self.clf_name)
        acc = self.clf.fit(self.X_train, self.y_train).score(self.X_test, self.y_test)
        y_pred = self.clf.predict(self.X_test)
        bal_acc = balanced_accuracy_score(self.y_test, y_pred)
        print(f"acc->{acc}, bal_acc->{bal_acc}")
        clf_score = pd.DataFrame(
            {self.clf_name: [acc, bal_acc]}, 
            index=['Accuracy', 'Balanced accuracy']
        )
        self.models_eval_scores = pd.concat([self.models_eval_scores, clf_score], axis=1).round(decimals=3)
        
        return self.models_eval_scores
    
    def plot_confusion_matrix(ncols=1,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
        
        fig, ax = plt.subplots()
        
        cm = confusion_matrix(self.y_test, self.y_hat)
        
        classes = self.y_train
        
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
       
        """ 
        print(cm)
        print('')

        ax.imshow(cm, interpolation='nearest', cmap=cmap)
        ax.set_title(title)
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.sca(ax)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else 'd' 
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            ax.text(j, i, format(cm[i, j], fmt),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")

        ax.set_ylabel('True label')
        ax.set_xlabel('Predicted label')
    
    def plot_roc(self, n_classes=2):
       # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(self.y_test[:, i],  self.y_hat[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(self.y_test, self.y_hat)
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        

                # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += interp(all_fpr, fpr[i], tpr[i])

        # Finally average it and compute AUC
        mean_tpr /= n_classes

        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

        # Plot all ROC curves
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]),
                 color='deeppink', linestyle=':', linewidth=4)

        plt.plot(fpr["macro"], tpr["macro"],
                 label='macro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["macro"]),
                 color='navy', linestyle=':', linewidth=4)

        colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
        for i, color in zip(range(n_classes), colors):
            plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                     label='ROC curve of class {0} (area = {1:0.2f})'
                     ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--', lw=lw)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()




### Test model_metrics.py codes

In [216]:
y_train = p.y_train.purchases.ravel()
y_test = p.y_test.purchases.ravel()

##### 1. Evaluating Model Performances
> put all models in a list and create Metrics object

In [217]:
models = [model1, model2, model3, model4, model5]
model_evaluations = Metrics(p.X_train, p.y_train.purchases.ravel(), p.X_test, p.y_test.purchases.ravel())

> Create an empty Dataframe for models evaluations

In [218]:
models_scores = pd.DataFrame()

> Loop through models and evaluate

In [219]:
for model in models:
    
    models_scores = model_evaluations.evaluate_classifier(clf=model, models_eval_scores = models_scores);

LogisticRegressionCV
acc->0.9112772180695483, bal_acc->0.8579548231446965
LogisticRegression


/home/patrick/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


acc->0.9119272018199545, bal_acc->0.8613726335245323
RandomForestClassifier
acc->0.9480012999675008, bal_acc->0.760407826863523
BalancedRandomForestClassifier
acc->0.9356516087097823, bal_acc->0.8246486664208184
SVC
acc->0.8716282092947676, bal_acc->0.8309344606812961


add model1

##### Comparison of Model Performances (Acuuracy Vs Balanced Accuracy)

In [221]:
models_scores.T

,Accuracy,Balanced accuracy
LogisticRegressionCV,0.911,0.858
LogisticRegression,0.912,0.861
RandomForestClassifier,0.948,0.760
BalancedRandomForestClassifier,0.936,0.825
SVC,0.872,0.831


#### 2. Test conf_matrix, accuracy_score, classification_error, specif_sensitiv

### Module pipeline.py Codes

In [ ]:
#%%writefile pipeline.py
#%%writefile ../scripts/project_package/model_package/pipeline.py 
from sklearn.pipeline import FeatureUnion, Pipeline  
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline, make_pipeline